In [1]:
# Import Libraries
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from scipy import stats 

In [2]:
# Import NASDAQ list of stocks
stocks = pd.read_csv('nas.csv')
stocks

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,FB
4,TSLA
...,...
96,VRSN
97,DOCU
98,SWKS
99,SGEN


In [3]:
# Import API token from secrets.py
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
# Make the First API call
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 96500321, 'calculationPrice': 'iexlasttrade', 'change': -0.1, 'changePercent': -0.00065, 'close': 0, 'closeSource': 'cfaoflii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': None, 'highTime': None, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 167.57, 'iexCloseTime': 1658595514987, 'iexLastUpdated': 1683478816046, 'iexMarketPercent': 0.02006924015425913, 'iexOpen': 170.84, 'iexOpenTime': 1703141959269, 'iexRealtimePrice': 162.18, 'iexRealtimeSize': 2, 'iexVolume': 2240927, 'lastTradeTime': 1646476084593, 'latestPrice': 166.76, 'latestSource': 'IEX Last Trade', 'latestTime': 'January 26, 2022', 'latestUpdate': 1646654545691, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap': 2729632031565, 'oddLotDelayedPrice

In [5]:
# Pass API call
price = data['latestPrice']
market_cap = data['marketCap']
print(market_cap/1000000000000)

2.729632031565


In [6]:
# Add stocks data to pandas DataFrame
my_columns = ["Ticker", "Stock Price", "Market Cap", "Numbers of shares to buy"]
final_dataframe = pd.DataFrame(columns = my_columns)

In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
    index=my_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,Numbers of shares to buy
0,AAPL,166.76,2729632031565,N/A


In [8]:
# Loop through the rest of the tickers
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)

In [9]:
final_dataframe.head()

,Ticker,Stock Price,Market Cap,Numbers of shares to buy
0,AAPL,162.400,2616702485718,N/A
1,MSFT,310.000,2302147269563,N/A
2,AMZN,2869.093,1458770458018,N/A
3,FB,297.500,842542069746,N/A
4,TSLA,964.288,953545823241,N/A


In [10]:
# Using Batch API calls
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'], 
                data[symbol]['quote']['marketCap'], 
                'N/A'
             ], 
            index = my_columns), 
            ignore_index = True
        )
        
final_dataframe      
    

,Ticker,Stock Price,Market Cap,Numbers of shares to buy
0,AAPL,167.400,2704846442287,N/A
1,MSFT,303.520,2242904623947,N/A
2,AMZN,2866.177,1435287968277,N/A
3,FB,302.050,866803235533,N/A
4,TSLA,966.419,952388603291,N/A
...,...,...,...,...
96,VRSN,212.000,23850843536,N/A
97,DOCU,120.060,23531071474,N/A
98,SWKS,148.220,24274942098,N/A
99,SGEN,128.090,23347562869,N/A


In [12]:
# Calculate numbers of shares to buy
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio: 15000


In [13]:
position_size = val / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Numbers of shares to buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Cap,Numbers of shares to buy
0,AAPL,167.400,2704846442287,0
1,MSFT,303.520,2242904623947,0
2,AMZN,2866.177,1435287968277,0
3,FB,302.050,866803235533,0
4,TSLA,966.419,952388603291,0
...,...,...,...,...
96,VRSN,212.000,23850843536,0
97,DOCU,120.060,23531071474,1
98,SWKS,148.220,24274942098,1
99,SGEN,128.090,23347562869,1


In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [18]:
writer.save()

C:\backup\envs\dev\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
